# ENIGMA SCOREBOARD  
for Tulane University Live Action Gaming  
10/12/2020 - 10/18/2020 

### Import Packages  
Here we will import the packages that we need to access Google Sheets.

In [267]:
# Install packages using sys
import sys
!{sys.executable} -m pip install gspread oauth2client df2gspread

# Import libraries
import pandas as pd
import numpy as np
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

### Importing Sheets  
Now, we'll work some magic to import data from the google sheets. See https://towardsdatascience.com/how-to-integrate-google-sheets-and-jupyter-notebooks-c469309aacea for the tutorial that I used.  
  
First, let's get our service account's credentials:

In [268]:
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('./enigma-scoreboard-tulag-281332669163.json', scope) 
gc = gspread.authorize(credentials)

Now, let's import the data from our Google sheet that contains player responses to puzzles!

In [269]:
spreadsheet_key = '1HNhbYW-OlHOqR48ycrhM4NOEQtK7CR1MGrsSJgDFVrc' 
book = gc.open_by_key(spreadsheet_key) 

In [270]:
# Now let's open up a the entire Scoresheet
scoresheet = gc.open_by_key(spreadsheet_key)
puzzle = scoresheet.get_worksheet(0) # Get sheet of responses from spreadsheet
puzzle_df = pd.DataFrame(puzzle.get_all_records()) # Convert to dataframe
puzzle_df

,Timestamp,Your Answer:,Your Name:
0,10/2/2020 16:44:23,test answer,test player
1,10/13/2020 14:03:20,Post,Benjamin Villalpando
2,10/13/2020 15:38:30,Post,Matthew Shernicoff
3,10/14/2020 14:14:03,Post,Sarah Smith
4,10/14/2020 23:32:09,Post,Jared Kessler
5,10/15/2020 13:39:59,Post,Benton Meldrum and Emma Brick-Hezeau
6,10/15/2020 15:14:23,Post,Courtney Zaharia


### Building the Scoreboard  
Now that we have access to the g-sheet of player responses, let's put these together into a DataFrame that tallies the number of responses that each player has submitted.

In [271]:
# Iterate over each sheet in the scoresheet and tally each player's total number of responses

players_df = pd.DataFrame(columns = ['Name', 'Total Responses'])

responses_list = [] # list of dataframe objects 
for sheet in scoresheet: # for each sheet of puzzle responses
    puzzle_df = pd.DataFrame(sheet.get_all_records()) # convert to dataframe
    players = puzzle_df['Your Name:'].unique() # players who have responded to the puzzle - use .unique to filter duplicate responses
    for player in players:
        # If player not yet in dataframe, add them 
        if player not in players_df['Name'].tolist():
            player = {'Name' : player, 'Total Responses' : 1}
            players_df = players_df.append(player, ignore_index = True)
        # If already in dataframe, update their score
        if player in players_df['Name'].tolist():
            idx = players_df[players_df['Name'] == player].index.item()
            players_df.at[idx, 'Total Responses'] += 1
            
        
players_df

,Name,Total Responses
0,test player,29
1,Benjamin Villalpando,11
2,Matthew Shernicoff,9
3,Sarah Smith,15
4,Jared Kessler,14
5,Benton Meldrum and Emma Brick-Hezeau,4
6,Courtney Zaharia,10
7,Liv G,1
8,Jared Kessler,1
9,Benton Meldrum and Emma Brick-Hezeau,3


### Cleaning Up the Scoreboard  
This scoreboard contains the data that we want, but you may notice that there are a few issues. First, the dummy responses used to test the Google Forms -> Sheets link need to be deleted.  

Second, some players show up more than once in the table. This is because the strings they entered into the Google Forms were not the same every time - there may be differences in spelling or whitespace. To fix this, we will need to identify duplicate players and combine their rows into one, tallying their score between the rows.

In [272]:
# First, let's delete the test entries in the table:
players_df = players_df.drop(players_df[players_df['Name'] == 'test answer'].index.item())
players_df = players_df.drop(players_df[players_df['Name'] == 'test player'].index.item())

Now let's try to solve for players appearing more than once. To do this, we will use Levenshtein distance. First, let's install the associated library:

In [273]:
!{sys.executable} -m pip install python-Levenshtein
import Levenshtein as lev

Levenshtein distance is defined as the number of characters that must be changed or dropped for one string to match another. For example, the Levenshtein distance between "Sarah" and "Sara" is one - as there is only one character's difference. 

In [274]:
lev.distance('Benton Meldrum and Emma Brick-Hezeau', 'Benton Meldrum and Emma Brick-Hezeau')

0

Let's say that 2 is an acceptable Lev distance to match players. We will need to do an n^2 operation, iterating through our df to compare player names.

In [275]:
matched = []
for name_x in players_df['Name']:
    for name_y in players_df['Name']:
        if name_y not in matched:
            if lev.distance(name_x, name_y) <= 2:
                if lev.distance(name_x, name_y) != 0:
                    print(name_x, name_y)
                    idx_x = players_df[players_df['Name'] == name_x].index.item() #index of player x
                    idx_y = players_df[players_df['Name'] == name_y].index.item() #index of player y
                    players_df.at[idx_x, 'Total Responses'] += players_df.at[idx_y, 'Total Responses'] # add scores together in row of name_x
                    matched.append(name_x)
                    print(matched)
                    players_df = players_df.drop(players_df[players_df['Name'] == name_y].index.item()) # drop row of name_y
players_df

Matthew Shernicoff Matthew Shernicoff 
['Matthew Shernicoff']
Sarah Smith Sarah Smith 
['Matthew Shernicoff', 'Sarah Smith']
Jared Kessler Jared Kessler 
['Matthew Shernicoff', 'Sarah Smith', 'Jared Kessler']
Benton Meldrum and Emma Brick-Hezeau Benton Meldrum and Emma Brick-Hezeau 
['Matthew Shernicoff', 'Sarah Smith', 'Jared Kessler', 'Benton Meldrum and Emma Brick-Hezeau']


,Name,Total Responses
1,Benjamin Villalpando,11
2,Matthew Shernicoff,16
3,Sarah Smith,16
4,Jared Kessler,15
5,Benton Meldrum and Emma Brick-Hezeau,7
6,Courtney Zaharia,10
7,Liv G,1
11,Mads RingswaldEgan,1
12,Lily Livaudais,2
13,Sarah Scism,1


It worked! Hooray!

Notably, this whole thing would break down if we have two people that just happen to share a name, i.e. two separate people named "Sarah Smith." This would've caused problems for our team even if we hadn't tried to make an automated scoreboard. If it ever becomes a problem, we'll fix it in a future iteration of the game.

Now let's clean up our dataframe a little bit to rank the players:

In [276]:
players_df = players_df.sort_values(by = 'Total Responses', ascending = False).reset_index()
del players_df['index']
players_df.index += 1

In [283]:
# do some datetime stuff
from datetime import datetime

# SCOREBOARD  
This is the most recent scoreboard for Enigma!  
  
A few notes:
* This scoreboard does NOT account for correctness of responses submitted. This only counts the number of puzzles that the player has responded to. Our team will check for correctness at the end of the game.
* In case of ties, at the end of the game, we will check to see which player solved their puzzles more quickly. This scoreboard does NOT rank players according to their most recent response; if there is a tie, their ranking is entirely abritrary. 

In [284]:
print("LAST UPDATED: ", datetime.now())
players_df

LAST UPDATED:  2020-10-16 22:22:38.087776


,Name,Total Responses
1,Matthew Shernicoff,16
2,Sarah Smith,16
3,Jared Kessler,15
4,Benjamin Villalpando,11
5,Courtney Zaharia,10
6,Benton Meldrum and Emma Brick-Hezeau,7
7,Lily Livaudais,2
8,Liv G,1
9,Mads RingswaldEgan,1
10,Sarah Scism,1
